In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("./belangrijkste-incidenten.csv", sep=";")

In [3]:
data.head()

,Mois,Date de l'incident,Ligne,Lieu,Lieu.1,Lieu.2,Description de l'incident NL,Description de l'incident,Description de l'incident EN,Nombre de minutes de retard,Nombre de trains supprimés
0,2025-09,2025-09-26,59,ANTWERPEN-ZUID,ANTWERPEN-ZUID,ANTWERPEN-ZUID,Spoorlopers,Intrusion dans les voies,Intrusion into the tracks,1051,29
1,2025-09,2025-09-22,161,BOSVOORDE,BOSVOORDE,BOSVOORDE,Obstakel in de nabijheid van het spoor,Obstacle dans/à proximité de la voie,Obstacle in/near the tracks,1588,74
2,2025-09,2025-09-17,161,MONT-SAINT-GUIBERT,MONT-SAINT-GUIBERT,MONT-SAINT-GUIBERT,Schade treinmaterieel,Avarie matériel roulant,Damage rolling stock,1083,37
3,2025-09,2025-09-15,60,DENDERMONDE,DENDERMONDE,DENDERMONDE,Storing seininrichting,Dérangement à la signalisation,Disturbance with signalling,1288,12
4,2025-09,2025-09-11,50A,BEERNEM,BEERNEM,BEERNEM,Persoonsongeval,Heurt d'une personne,Collision with a person,3755,70


In [4]:
# Supprimer les lignes où la colonne "Ligne" vaut "-"
data = data[data["Ligne"] != "-"]

In [5]:
# Convertir la colonne 'Date de l'incident' en datetime
data["Date de l'incident"] = pd.to_datetime(data["Date de l'incident"], errors="coerce")

In [6]:
# Extraire le mois sous forme de chaîne en français
data["Mois"] = pd.to_datetime(data["Mois"], errors="coerce").dt.month

mois_fr = {
    1: "janvier", 2: "février", 3: "mars", 4: "avril",
    5: "mai", 6: "juin", 7: "juillet", 8: "août",
    9: "septembre", 10: "octobre", 11: "novembre", 12: "décembre"
}

data["Mois"] = data["Mois"].map(mois_fr)

In [7]:
data = data[['Mois', "Date de l'incident", 'Ligne', 'Lieu', "Description de l'incident", 'Nombre de minutes de retard', 'Nombre de trains supprimés']]

In [8]:
# Ajout d'une colonne saison
def saison(mois):
    saisons = {
        "décembre": "hiver", "janvier": "hiver", "février": "hiver",
        "mars": "printemps", "avril": "printemps", "mai": "printemps",
        "juin": "été", "juillet": "été", "août": "été",
        "septembre": "automne", "octobre": "automne", "novembre": "automne"
    }
    return saisons.get(mois, None)

data["Saison"] = data["Mois"].apply(saison)

In [9]:
data.head()

,Mois,Date de l'incident,Ligne,Lieu,Description de l'incident,Nombre de minutes de retard,Nombre de trains supprimés,Saison
0,septembre,2025-09-26,59,ANTWERPEN-ZUID,Intrusion dans les voies,1051,29,automne
1,septembre,2025-09-22,161,BOSVOORDE,Obstacle dans/à proximité de la voie,1588,74,automne
2,septembre,2025-09-17,161,MONT-SAINT-GUIBERT,Avarie matériel roulant,1083,37,automne
3,septembre,2025-09-15,60,DENDERMONDE,Dérangement à la signalisation,1288,12,automne
4,septembre,2025-09-11,50A,BEERNEM,Heurt d'une personne,3755,70,automne


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 801 entries, 0 to 833
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Mois                         801 non-null    object        
 1   Date de l'incident           801 non-null    datetime64[ns]
 2   Ligne                        801 non-null    object        
 3   Lieu                         801 non-null    object        
 4   Description de l'incident    801 non-null    object        
 5   Nombre de minutes de retard  801 non-null    int64         
 6   Nombre de trains supprimés   801 non-null    int64         
 7   Saison                       801 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 56.3+ KB


In [11]:
data.describe()

,Date de l'incident,Nombre de minutes de retard,Nombre de trains supprimés
count,801,801.000000,801.000000
mean,2022-04-28 03:37:31.685393152,1920.024969,44.739076
min,2019-01-03 00:00:00,0.000000,0.000000
25%,2020-10-01 00:00:00,1150.000000,18.000000
50%,2022-06-28 00:00:00,1440.000000,30.000000
75%,2023-09-25 00:00:00,2055.000000,54.000000
max,2025-09-26 00:00:00,24757.000000,1080.000000
std,NaN,1554.328380,58.835581


In [12]:
data.isna().sum()

Mois                           0
Date de l'incident             0
Ligne                          0
Lieu                           0
Description de l'incident      0
Nombre de minutes de retard    0
Nombre de trains supprimés     0
Saison                         0
dtype: int64

In [13]:
for col in data.columns:
    print(f"{col}: {data[col].nunique()} valeurs uniques")

Mois: 12 valeurs uniques
Date de l'incident: 634 valeurs uniques
Ligne: 83 valeurs uniques
Lieu: 272 valeurs uniques
Description de l'incident: 33 valeurs uniques
Nombre de minutes de retard: 621 valeurs uniques
Nombre de trains supprimés: 144 valeurs uniques
Saison: 4 valeurs uniques


In [22]:
def normalize(col):
    return ((col - col.min()) / (col.max() - col.min()) * 100)

data["norm_retards"] = normalize(data["Nombre de minutes de retard"])
data["norm_suppressions"] = normalize(data["Nombre de trains supprimés"])

In [23]:
data.to_csv('data.csv', index=False)

In [24]:
data.to_excel('data.xlsx', index=False)